In [1]:
using Turing
using LimberJack
using CSV
using NPZ
using FITSIO
using LinearAlgebra
using PythonCall
using ForwardDiff
np = pyimport("numpy")

Python module: <module 'numpy' from '/home/jaime/.julia/environments/v1.7/.CondaPkg/env/lib/python3.10/site-packages/numpy/__init__.py'>

In [2]:
#println("My id is ", myid(), " and I have ", Threads.nthreads(), " threads")

fol = "DESY1"
data_set = "wlwl_Nzs"
meta = np.load(string("../data/", fol, "/", data_set, "_meta.npz"))
files = npzread(string("../data/", fol, "/", data_set, "_files.npz"))

tracers_names = pyconvert(Vector{String}, meta["tracers"])
pairs = pyconvert(Vector{Vector{String}}, meta["pairs"]);
idx = pyconvert(Vector{Int}, meta["idx"])
data_vector = pyconvert(Vector{Float64}, meta["cls"])
cov_tot = pyconvert(Matrix{Float64}, meta["cov"])
inv_cov_tot = pyconvert(Matrix{Float64}, meta["inv_cov"])

nz_path = "../data/DESY1/Nzs/"
zs_k0, nz_k0, cov_k0 = get_nzs(nz_path, "DESwl__0_e")
zs_k1, nz_k1, cov_k1 = get_nzs(nz_path, "DESwl__1_e")
zs_k2, nz_k2, cov_k2 = get_nzs(nz_path, "DESwl__2_e")
zs_k3, nz_k3, cov_k3 = get_nzs(nz_path, "DESwl__3_e")

(Float32[0.06, 0.099999994, 0.14, 0.18, 0.22, 0.26, 0.3, 0.34, 0.38, 0.42000002  …  1.26, 1.3, 1.34, 1.38, 1.4200001, 1.46, 1.5, 1.54, 1.5799999, 1.62], [4.629577847407024, 29.132412608452444, 21.765409190156518, 20.62146040771289, 49.575919814294636, 19.756458131549834, 28.671379304807797, 16.733425194984918, 36.12801036345403, 15.65029828338953  …  159.1343087971742, 45.68244851651424, 57.13816009303667, 74.52981085428384, 67.0412204252083, 69.0824797219035, 38.605059715930935, 18.387277685421846, 18.004397396845302, 19.840074411368867], [34.72926020369569 -0.3508402184688245 … -0.0035034826825542913 0.025896520365390033; -0.3508402184688245 527.8777602661406 … -0.09997382746286836 -0.09250459543441937; … ; -0.0035034826825542913 -0.09997382746286836 … 94.73249576839464 0.09124693778804871; 0.025896520365390033 -0.09250459543441937 … 0.09124693778804871 106.16024698844336])

In [3]:
isposdef(cov_k0)

true

## T-Matrices

In [4]:
function model_cov0(x;
               tracers_names=tracers_names,
               pairs=pairs,
               idx=idx,
               cov_tot=cov_tot, 
               files=files) 

    cosmology = Cosmology()
    #cosmology.settings.cosmo_type = Real
    nuisances = Dict("A_IA" => 0,
                     "alpha_IA" => 0,

                     "DESwl__0_e_nz" => x,
                     "DESwl__1_e_nz" => nz_k1,
                     "DESwl__2_e_nz" => nz_k2,
                     "DESwl__3_e_nz" => nz_k3,
        
                     "DESwl__0_e_m" => 0.012,
                     "DESwl__1_e_m" => 0.012,
                     "DESwl__2_e_m" => 0.012,
                     "DESwl__3_e_m" => 0.012)
    
    return Theory(cosmology, tracers_names, pairs,
                  idx, files; Nuisances=nuisances)
end;

function model_cov1(x;
               tracers_names=tracers_names,
               pairs=pairs,
               idx=idx,
               cov_tot=cov_tot, 
               files=files) 

    cosmology = Cosmology()
    #cosmology.settings.cosmo_type = Real
    nuisances = Dict("A_IA" => 0,
                     "alpha_IA" => 0,

                     "DESwl__0_e_nz" => nz_k0,
                     "DESwl__1_e_nz" => x,
                     "DESwl__2_e_nz" => nz_k2,
                     "DESwl__3_e_nz" => nz_k3,
        
                     "DESwl__0_e_m" => 0.012,
                     "DESwl__1_e_m" => 0.012,
                     "DESwl__2_e_m" => 0.012,
                     "DESwl__3_e_m" => 0.012)
    
    return Theory(cosmology, tracers_names, pairs,
                  idx, files; Nuisances=nuisances)
end;

function model_cov2(x;
               tracers_names=tracers_names,
               pairs=pairs,
               idx=idx,
               cov_tot=cov_tot, 
               files=files) 

    cosmology = Cosmology()
    #cosmology.settings.cosmo_type = Real
    nuisances = Dict("A_IA" => 0,
                     "alpha_IA" => 0,

                     "DESwl__0_e_nz" => nz_k0,
                     "DESwl__1_e_nz" => nz_k1,
                     "DESwl__2_e_nz" => x,
                     "DESwl__3_e_nz" => nz_k3,
        
                     "DESwl__0_e_m" => 0.012,
                     "DESwl__1_e_m" => 0.012,
                     "DESwl__2_e_m" => 0.012,
                     "DESwl__3_e_m" => 0.012)
    
    return Theory(cosmology, tracers_names, pairs,
                  idx, files; Nuisances=nuisances)
end;

function model_cov3(x;
               tracers_names=tracers_names,
               pairs=pairs,
               idx=idx,
               cov_tot=cov_tot, 
               files=files) 

    cosmology = Cosmology()
    #cosmology.settings.cosmo_type = Real
    nuisances = Dict("A_IA" => 0,
                     "alpha_IA" => 0,

                     "DESwl__0_e_nz" => nz_k0,
                     "DESwl__1_e_nz" => nz_k1,
                     "DESwl__2_e_nz" => nz_k2,
                     "DESwl__3_e_nz" => x,
        
                     "DESwl__0_e_m" => 0.012,
                     "DESwl__1_e_m" => 0.012,
                     "DESwl__2_e_m" => 0.012,
                     "DESwl__3_e_m" => 0.012)
    
    return Theory(cosmology, tracers_names, pairs,
                  idx, files; Nuisances=nuisances)
end;

In [5]:
d = 0.0005
m  = length(nz_k0)
T0 = zeros(240, m)
T1 = zeros(240, m)
T2 = zeros(240, m)
T3 = zeros(240, m)

for i in 1:m
    e = zeros(m)
    e[i] = d
    T0[:, i] = (model_cov0(nz_k0+e) .- model_cov0(nz_k0)) ./ d
    T1[:, i] = (model_cov1(nz_k1+e) .- model_cov1(nz_k1)) ./ d
    T2[:, i] = (model_cov2(nz_k2+e) .- model_cov2(nz_k2)) ./ d
    T3[:, i] = (model_cov3(nz_k3+e) .- model_cov3(nz_k3)) ./ d
end

In [6]:
#T0 = ForwardDiff.jacobian(model_cov0, nz_k0)
#T1 = ForwardDiff.jacobian(model_cov1, nz_k1)
#T2 = ForwardDiff.jacobian(model_cov2, nz_k2)
#T3 = ForwardDiff.jacobian(model_cov3, nz_k3)

In [7]:
TT0 = T0 * cov_k0 * T0'
TT1 = T1 * cov_k1 * T1'
TT2 = T2 * cov_k2 * T2'
TT3 = T3 * cov_k3 * T3';

In [29]:
Inf

Inf

In [13]:
C_mod = Symmetric(cov_tot + TT0 + TT1 + TT2 + TT3);
inv_C_mod = inv(C_mod)

240×240 Symmetric{Float64, Matrix{Float64}}:
  1.26529e18   1.55945e17  -9.61225e15  …  -1.91867e15  -2.28142e15
  1.55945e17   2.62924e18   2.90029e17     -1.05757e15  -1.25754e15
 -9.61225e15   2.90029e17   4.00969e18     -8.99727e14  -1.0711e15
 -1.05699e16  -8.00169e15   4.35568e17     -8.2682e14   -9.88956e14
 -9.31186e15  -1.32386e16   5.22754e15     -7.6988e14   -9.26354e14
 -8.03296e15  -1.40389e16  -7.57025e15  …  -7.18897e14  -8.68718e14
 -7.72421e15  -1.44817e16  -1.26892e16     -6.72533e14  -8.14389e14
 -7.53437e15  -1.48162e16  -1.57583e16     -6.71619e14  -8.14376e14
 -8.13045e15  -1.6401e16   -1.81047e16     -7.22641e14  -8.79452e14
 -8.46326e15  -1.74163e16  -2.03623e16     -7.51966e14  -9.32154e14
 -9.03695e15  -1.86508e16  -2.20970e16  …  -7.69497e14  -9.61132e14
 -1.01602e16  -2.10542e16  -2.56092e16     -8.02649e14  -1.00564e15
 -1.11795e16  -2.31552e16  -2.85696e16     -8.15672e14  -1.02666e15
  ⋮                                     ⋱               
 -2.28555e14   

In [19]:
abs.(cov_tot - C_mod) / cov_tot

240×240 Matrix{Float64}:
 0.00626303   0.0202006    0.0233206    …  0.000723988  0.000977839
 0.00277358   0.00950408   0.0110649       0.00035426   0.000478601
 0.00150077   0.00533352   0.00623926      0.000203236  0.000274609
 0.000917572  0.00334265   0.00392259      0.000129212  0.000174607
 0.000609051  0.00225905   0.00265689      8.82124e-5   0.000119211
 0.000428493  0.00161167   0.00189872   …  6.34171e-5   8.57068e-5
 0.000314814  0.00119758   0.00141279      4.74117e-5   6.40784e-5
 0.00023717   0.000910996  0.00107593      3.62521e-5   4.89976e-5
 0.000178907  0.000693512  0.000819951     2.77302e-5   3.74807e-5
 0.000134168  0.000524518  0.000620755     2.10652e-5   2.84729e-5
 0.000100435  0.00039585   0.000468917  …  1.59641e-5   2.15787e-5
 7.48861e-5   0.000297497  0.000352726     1.20458e-5   1.62827e-5
 5.56358e-5   0.000222645  0.000264196     9.04799e-6   1.22308e-5
 ⋮                                      ⋱               
 4.62776e-5   3.03763e-5   2.28558e-5     

In [15]:
npzwrite("../data/DESY1/Nzs/cov_marg.npz",
        Dict("TPT0" => TT0, "TPT1" => TT1,
             "TPT2" => TT2, "TPT3" => TT3,
             "cov_marg" => C_mod,
             "inv_cov_marg" => inv_C_mod))

## Likelihood

In [3]:
pars = [4.426868e-02,     2.093138e-01,     8.963611e-01,     8.495440e-01,
         1.343888e+00,    1.639047e+00,      1.597174e+00,     1.944583e+00,     2.007245e+00,
        -4.679383e-03,   -2.839996e-03,      1.771571e-03,     1.197051e-03,    -5.199799e-03,
         2.389208e-01,   -6.435288e-01, 
         1.802722e-03,   -5.508994e-03,     1.952514e-02,    -1.117726e-03,
        -1.744083e-02,    6.777779e-03,    -1.097939e-03,    -4.912315e-03,
         8.536883e-01,    2.535825e-01];

In [4]:
function get_cls(Ωm, s8;
               tracers_names=tracers_names,
               pairs=pairs,
               idx=idx,
               cov_tot=cov_tot, 
               files=files) 

    Ωb = pars[1]
    h = pars[4]
    ns = pars[3]

    cosmology = LimberJack.Cosmology(Ωm, Ωb, h, ns, s8, 
                                     tk_mode="EisHu",
                                     Pk_mode="Halofit")
    nuisances = Dict("A_IA" => 0,
                     "alpha_IA" => 0,

                     "DESwl__0_e_nz" => nz_k0,
                     "DESwl__1_e_nz" => nz_k1,
                     "DESwl__2_e_nz" => nz_k2,
                     "DESwl__3_e_nz" => nz_k3,
        
                     "DESwl__0_e_m" => 0.012,
                     "DESwl__1_e_m" => 0.012,
                     "DESwl__2_e_m" => 0.012,
                     "DESwl__3_e_m" => 0.012)
    
    return Theory(cosmology, tracers_names, pairs,
                  idx, files; Nuisances=nuisances)
end;


In [5]:
function get_Xi2(Ωm, s8;
                 data_vector=data_vector,
                 inv_cov=inv_cov_tot)

        nuisances = Dict("A_IA" => 0,
                     "alpha_IA" => 0,

                     "DESwl__0_e_nz" => nz_k0,
                     "DESwl__1_e_nz" => nz_k1,
                     "DESwl__2_e_nz" => nz_k2,
                     "DESwl__3_e_nz" => nz_k3,
        
                     "DESwl__0_e_m" => 0.012,
                     "DESwl__1_e_m" => 0.012,
                     "DESwl__2_e_m" => 0.012,
                     "DESwl__3_e_m" => 0.012)
    

    Ωb = pars[1]
    h = pars[5]
    ns = pars[4]
    
    cosmology = LimberJack.Cosmology(Ωm, Ωb, h, ns, s8, 
                                     tk_mode="EisHu",
                                     Pk_mode="Halofit")
    
    theory = Theory(cosmology, tracers_names, pairs,
                    idx, files; Nuisances=nuisances)
    
    diff = @.(theory-data_vector)
    return dot(diff, inv_cov * diff)

end

get_Xi2 (generic function with 1 method)

In [6]:
Ωms = LinRange(0.2, 0.6, 10)
s8s = LinRange(0.6, 0.9, 10)

10-element LinRange{Float64, Int64}:
 0.6,0.633333,0.666667,0.7,0.733333,0.766667,0.8,0.833333,0.866667,0.9

In [7]:
get_Xi2(pars[26], pars[25])

255.52773157235953

In [44]:
Xi2s_emul = zeros(length(Ωms), length(s8s))
Xi2s_EisHu = zeros(length(Ωms), length(s8s))
for i in 1:length(Ωms)
    for j in 1:length(s8s)
        Xi2s_emul[i, j] = get_Xi2(Ωms[i], s8s[j]; tk_mode="emulator")
        Xi2s_EisHu[i, j] = get_Xi2(Ωms[i], s8s[j]; tk_mode="EisHu")
    end
end

LoadError: UndefVarError: Ωms not defined

In [8]:
n_samples = 100
cls = zeros(240, n_samples)
for i in 1:n_samples
    Ωm = rand(Uniform(0.2, 0.6))
    s8 = rand(Uniform(0.6, 0.9))
    cls[:, i] = get_cls(Ωm, s8)
end

In [9]:
npzwrite("Nz_cls.npz", Dict("cls" => cls))

## Turing

In [46]:
chain = sample(model(fake_data), NUTS(adaptation, TAP, ),
               MCMCDistributed(), iterations, nchains, progress=true; save_state=true)

LoadError: UndefVarError: fake_data not defined